### Predicting stock price movements: Part 1 of 2

This program creates a data set to be used in machine learning models
to predict the future direction of stock prices. The general approach used in prediction is based on a paper by Jim Kyung-Soo Liew and Boris Mayster: "Forecasting ETFs with Machine Learning Algorithms" (2017). 

This program gets the raw data from yahoo, transforms it into a dataset that can directly be fed into scikit-learn for prediction, and writes the transformed dataset in a CSV file.


The inputs to the program are (1) set of ticker symbols, and (2) timeframe, i.e, the beginning and end dates. We also need N (number of days to be used to calculate returns and average volumes) and j (number of lags of N-day returns and average volumes). In this program, N = 20 and j = 10. The same time frame will be used for all tickers and all tickers need to have data for the specified timeframe to avoid issues related to missing data.

Initial version: February 21, 2017 (APPL data only)
Next revision: June 22, 2017  (Generalizes to a set of tickers)
This version: July 7, 2017 - creates an output in "wide" format so that for each day, there are a number of columns per Ticker. The ticker is part of the column name so that it's easy to identify tickers.

Note: yahoo data portal may not be available in the future. I had to do the "workaround" below - fix_yahoo_finance to get it to work in July 2017

Second note: yahoo finance API no longer sems to be available (as of early 2018). Quandl.com has similar publicly-available data for stocks (but not for ETFs).

Murat Aydogdu

In [1]:
from IPython.display import display
import pandas as pd
import numpy as np
import datetime

In [2]:
pd.set_option('display.max_columns', None)
#pd.options.display.max_rows = 1000
pd.options.display.float_format = '{:20,.2f}'.format

In [65]:
from pandas_datareader import data as pdr
import fix_yahoo_finance  # This is the "temporary" fix to get the data from yahoo
tickers = ["AAPL", "GOOGL","AMZN","MSFT","IBM"]

dt = pdr.get_data_yahoo(tickers, start="1999-12-01", end="2017-06-30")

[*********************100%***********************]  5 of 5 downloaded

In [66]:
# This is panel data, not a data frame
dt

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 4424 (major_axis) x 5 (minor_axis)
Items axis: Open to Volume
Major_axis axis: 1999-12-01 00:00:00 to 2017-06-30 00:00:00
Minor_axis axis: AAPL to MSFT

In [67]:
# Convert panel to data frame
dtdf = dt.to_frame()
display(dtdf)

Open                 High  \
Date       minor                                             
1999-12-01 AAPL                  3.61                 3.73   
           AMZN                 87.25                87.88   
           IBM                 102.56               104.44   
           MSFT                 45.53                46.97   
1999-12-02 AAPL                  3.68                 3.95   
           AMZN                 86.00                91.31   
           IBM                 103.44               106.31   
           MSFT                 46.53                47.62   
1999-12-03 AAPL                  4.01                 4.13   
           AMZN                 92.50                93.38   
           IBM                 109.81               112.88   
           MSFT                 47.91                48.56   
1999-12-06 AAPL                  4.09                 4.19   
           AMZN                 86.88                89.88   
           IBM                 113.00               116.50   
           MSFT                 47.62                48.59   
1999-12-07 AAPL                  4.16                 4.21   
           AMZN                 88.00                88.00   
           IBM                 117.00               119.19   
           MSFT                 47.38                47.44   
1999-12-08 AAPL                  4.15                 4.21   
           AMZN                 86.81                93.00   
           IBM                 116.25               121.00   
           MSFT                 46.56                47.16   
1999-12-09 AAPL                  3.96                 3.96   
           AMZN                 95.50               113.00   
           IBM                 120.50               122.12   
           MSFT                 46.00                46.66   
1999-12-10 AAPL                  3.76                 3.90   
           AMZN                111.94               112.00   
...                               ...                  ...   
2017-06-23 AAPL                145.13               147.16   
           AMZN              1,002.54             1,004.62   
           GOOGL               975.50               986.62   
           IBM                 154.34               154.68   
           MSFT                 70.09                71.25   
2017-06-26 AAPL                147.17               148.28   
           AMZN              1,008.50             1,009.80   
           GOOGL               990.00               993.99   
           IBM                 154.71               155.75   
           MSFT                 71.40                71.71   
2017-06-27 AAPL                145.01               146.16   
           AMZN                990.69               998.80   
           GOOGL               961.60               967.22   
           IBM                 155.00               155.34   
           MSFT                 70.11                70.18   
2017-06-28 AAPL                144.49               146.11   
           AMZN                978.55               990.68   
           GOOGL               950.66               963.24   
           IBM                 155.15               155.55   
           MSFT                 69.21                69.84   
2017-06-29 AAPL                144.71               145.13   
           AMZN                979.00               987.56   
           GOOGL               951.35               951.66   
           IBM                 155.35               155.74   
           MSFT                 69.38                69.49   
2017-06-30 AAPL                144.45               144.96   
           AMZN                980.12               983.47   
           GOOGL               943.99               945.00   
           IBM                 154.28               154.50   
           MSFT                 68.78                69.38   

                                  Low                Close  \
Date       minor                                             
19

In [22]:
# Save raw data to a CSV first
#dtdf.to_csv('RAWDATA.CSV')
# March 7, 2018: yahoo finance is no longer working.
# Similar data is available on quandl for common stocks.
# Use the existing raw data file from a previous run of the 
# data acquisition step above.
dtdf = pd.read_csv('RAWDATA.CSV')

In [23]:
# Transform and keep only adjusted prices and dollar volume
# Use dollar volume so that we can compare across securities
dtdf['P'] = dtdf['Adj Close']
dtdf['V'] = dtdf['Volume'] * dtdf['Close'] 
display(dtdf)

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,P,V
0,1999-12-01,AAPL,3.61,3.73,3.57,3.32,3.68,"154,641,200.00",3.68,"514,098,793.03"
1,1999-12-01,AMZN,87.25,87.88,81.97,85.00,85.00,"10,663,600.00",85.00,"906,406,000.00"
2,1999-12-01,IBM,102.56,104.44,102.25,77.49,103.42,"5,336,400.00",103.42,"413,506,877.82"
3,1999-12-01,MSFT,45.53,46.97,45.44,31.27,46.59,"48,864,200.00",46.59,"1,527,855,705.25"
4,1999-12-02,AAPL,3.68,3.95,3.63,3.55,3.94,"141,839,600.00",3.94,"504,139,213.72"
5,1999-12-02,AMZN,86.00,91.31,85.62,89.06,89.06,"9,538,700.00",89.06,"849,540,468.75"
6,1999-12-02,IBM,103.44,106.31,103.38,78.87,105.27,"6,216,900.00",105.27,"490,323,409.10"
7,1999-12-02,MSFT,46.53,47.62,46.44,31.81,47.41,"55,473,800.00",47.41,"1,764,767,363.15"
8,1999-12-03,AAPL,4.01,4.13,4.00,3.71,4.11,"161,980,000.00",4.11,"600,869,021.48"
9,1999-12-03,AMZN,92.50,93.38,86.06,86.56,86.56,"11,151,200.00",86.56,"965,275,750.00"


In [ ]:
# March 7, 2018: When starting with the flatfile (RAWDATA.CSV), this step is not needed
# Rename indices Date: level 0 and Ticker: level 1
#dtdf.index = dtdf.index.set_names('Date', level=0) 
#dtdf.index = dtdf.index.set_names('Ticker', level=1)

# Drop columns other than adjusted close and dollar volume
#dtdf.drop(dtdf.columns[[0,1,2,3,4,5]], inplace = True, axis=1)
# Sort by ticker
#dtdf.sort_index(level=1, ascending = True, inplace=True)
#display(dtdf)

In [24]:
# Starting with flatfile:
# Drop columns other than Date, Ticker, Adjusted closing price and Dollar volume
dtdf.drop(dtdf.columns[[2,3,4,5,6,7]], inplace = True, axis=1)

In [25]:
dtdf.sort_values(['Ticker', 'Date'], ascending=[True, True], inplace=True)
display(dtdf)

,Date,Ticker,P,V
0,1999-12-01,AAPL,3.68,"514,098,793.03"
4,1999-12-02,AAPL,3.94,"504,139,213.72"
8,1999-12-03,AAPL,4.11,"600,869,021.48"
12,1999-12-06,AAPL,4.14,"436,649,612.25"
16,1999-12-07,AAPL,4.21,"422,797,351.29"
20,1999-12-08,AAPL,3.93,"365,987,679.69"
24,1999-12-09,AAPL,3.76,"725,854,559.39"
28,1999-12-10,AAPL,3.68,"529,731,959.78"
32,1999-12-13,AAPL,3.54,"423,097,228.19"
36,1999-12-14,AAPL,3.39,"333,479,975.37"


Construct variables using lags and leads.

Today is day *t*. For each day *t*, we need P~t~, P~t-N~, P~t+N~, and AV~t~
 
 AV~t~ is the average dollar volume of last N days, ending (i.e., including) today. This is parallel to how returns are calculated.

In [26]:
# m : minus, p: plus
# Negative values can be used for leads ("forward lags")
# Return will be measured based on Pt and Pt-N
# Average dollar volume will be measured based on Vt through Vt-N

N = 20
lagN = N*1
leadN = N*-1

# Price: we just need the N-lead and N-lag values
cname = 'P'+'m'+str(N).zfill(2)
dtdf[cname] = dtdf.groupby('Ticker')['P'].shift(lagN)
cname = 'P'+'p'+str(N).zfill(2)
dtdf[cname] = dtdf.groupby('Ticker')['P'].shift(leadN)
cname = 'AV'
# Average dollar volume of last N days, ending (including) today
dtdf[cname] = dtdf.groupby('Ticker')['V'].rolling(lagN).mean().reset_index(0,drop=True)    
display(dtdf)

,Date,Ticker,P,V,Pm20,Pp20,AV
0,1999-12-01,AAPL,3.68,"514,098,793.03",nan,3.58,nan
4,1999-12-02,AAPL,3.94,"504,139,213.72",nan,3.67,nan
8,1999-12-03,AAPL,4.11,"600,869,021.48",nan,4.00,nan
12,1999-12-06,AAPL,4.14,"436,649,612.25",nan,3.66,nan
16,1999-12-07,AAPL,4.21,"422,797,351.29",nan,3.71,nan
20,1999-12-08,AAPL,3.93,"365,987,679.69",nan,3.39,nan
24,1999-12-09,AAPL,3.76,"725,854,559.39",nan,3.55,nan
28,1999-12-10,AAPL,3.68,"529,731,959.78",nan,3.49,nan
32,1999-12-13,AAPL,3.54,"423,097,228.19",nan,3.31,nan
36,1999-12-14,AAPL,3.39,"333,479,975.37",nan,3.11,nan


In [27]:
# Compute the N-day return 
#  and the sign of the N-period ahead price movement

dtdf['R'] = (dtdf['P'] / dtdf['Pm20']) - 1
dtdf['Y'] = (dtdf['Pp20'] >= dtdf['P']).astype(int)

dtdf = dtdf.dropna()
display(dtdf)

,Date,Ticker,P,V,Pm20,Pp20,AV,R,Y
80,1999-12-30,AAPL,3.58,"167,566,808.43",3.68,3.63,"363,007,272.51",-0.03,1
84,1999-12-31,AAPL,3.67,"135,815,743.06",3.94,3.71,"344,591,098.97",-0.07,1
88,2000-01-03,AAPL,4.00,"483,655,734.41",4.11,3.58,"338,730,434.62",-0.03,0
92,2000-01-04,AAPL,3.66,"423,520,692.32",4.14,3.53,"338,073,988.62",-0.12,0
96,2000-01-05,AAPL,3.71,"652,759,257.04",4.21,3.69,"349,572,083.91",-0.12,0
100,2000-01-06,AAPL,3.39,"588,342,234.14",3.93,3.86,"360,689,811.63",-0.14,1
104,2000-01-07,AAPL,3.55,"369,687,177.83",3.76,4.07,"342,881,442.56",-0.05,1
108,2000-01-10,AAPL,3.49,"398,128,945.80",3.68,4.10,"336,301,291.86",-0.05,1
112,2000-01-11,AAPL,3.31,"330,257,970.38",3.54,4.02,"331,659,328.97",-0.06,1
116,2000-01-12,AAPL,3.11,"686,270,313.02",3.39,4.05,"349,298,845.85",-0.08,1


In [28]:
pd.options.mode.chained_assignment = None  # default='warn'
# j-lags of returns and average volumes
J = 10
vars = ['R','AV']
for i in vars:
    for j in range (1,J+1):
        cname = i+str(j).zfill(2)
        dtdf[cname] = dtdf[i].shift(j)
display(dtdf)  

,Date,Ticker,P,V,Pm20,Pp20,AV,R,Y,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,AV01,AV02,AV03,AV04,AV05,AV06,AV07,AV08,AV09,AV10
80,1999-12-30,AAPL,3.58,"167,566,808.43",3.68,3.63,"363,007,272.51",-0.03,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
84,1999-12-31,AAPL,3.67,"135,815,743.06",3.94,3.71,"344,591,098.97",-0.07,1,-0.03,nan,nan,nan,nan,nan,nan,nan,nan,nan,"363,007,272.51",nan,nan,nan,nan,nan,nan,nan,nan,nan
88,2000-01-03,AAPL,4.00,"483,655,734.41",4.11,3.58,"338,730,434.62",-0.03,0,-0.07,-0.03,nan,nan,nan,nan,nan,nan,nan,nan,"344,591,098.97","363,007,272.51",nan,nan,nan,nan,nan,nan,nan,nan
92,2000-01-04,AAPL,3.66,"423,520,692.32",4.14,3.53,"338,073,988.62",-0.12,0,-0.03,-0.07,-0.03,nan,nan,nan,nan,nan,nan,nan,"338,730,434.62","344,591,098.97","363,007,272.51",nan,nan,nan,nan,nan,nan,nan
96,2000-01-05,AAPL,3.71,"652,759,257.04",4.21,3.69,"349,572,083.91",-0.12,0,-0.12,-0.03,-0.07,-0.03,nan,nan,nan,nan,nan,nan,"338,073,988.62","338,730,434.62","344,591,098.97","363,007,272.51",nan,nan,nan,nan,nan,nan
100,2000-01-06,AAPL,3.39,"588,342,234.14",3.93,3.86,"360,689,811.63",-0.14,1,-0.12,-0.12,-0.03,-0.07,-0.03,nan,nan,nan,nan,nan,"349,572,083.91","338,073,988.62","338,730,434.62","344,591,098.97","363,007,272.51",nan,nan,nan,nan,nan
104,2000-01-07,AAPL,3.55,"369,687,177.83",3.76,4.07,"342,881,442.56",-0.05,1,-0.14,-0.12,-0.12,-0.03,-0.07,-0.03,nan,nan,nan,nan,"360,689,811.63","349,572,083.91","338,073,988.62","338,730,434.62","344,591,098.97","363,007,272.51",nan,nan,nan,nan
108,2000-01-10,AAPL,3.49,"398,128,945.80",3.68,4.10,"336,301,291.86",-0.05,1,-0.05,-0.14,-0.12,-0.12,-0.03,-0.07,-0.03,nan,nan,nan,"342,881,442.56","360,689,811.63","349,572,083.91","338,073,988.62","338,730,434.62","344,591,098.97","363,007,272.51",nan,nan,nan
112,2000-01-11,AAPL,3.31,"330,257,970.38",3.54,4.02,"331,659,328.97",-0.06,1,-0.05,-0.05,-0.14,-0.12,-0.12,-0.03,-0.07,-0.03,nan,nan,"336,301,291.86","342,881,442.56","360,689,811.63","349,572,083.91","338,073,988.62","338,730,434.62","344,591,098.97","363,007,272.51",nan,nan
116,2000-01-12,AAPL,3.11,"686,270,313.02",3.39,4.05,"349,298,845.85",-0.08,1,-0.06,-0.05,-0.05,-0.14,-0.12,-0.12,-0.03,-0.07,-0.03,nan,"331,659,328.97","336,301,291.86","342,881,442.56","360,689,811.63","349,572,083.91","338,073,988.62","338,730,434.62","344,591,098.97","363,007,272.51",nan


In [29]:
dtdf = dtdf.dropna()

In [30]:
# All we need are the returns and volumes, 
# plus the target variable Y (future N-day price movement)

# July 14: We want the latest return R, and 10 of its lags
# and same for AV. So keep R and AV
# There will be 11 return / volume features 
# dtdf = dtdf.drop(['P','V','Pm20','Pp20', 'AV', 'R'], 1)
dtdf = dtdf.drop(['P','V','Pm20','Pp20'], 1)

In [31]:
columns = dtdf.columns.tolist()
columns

['Date',
 'Ticker',
 'AV',
 'R',
 'Y',
 'R01',
 'R02',
 'R03',
 'R04',
 'R05',
 'R06',
 'R07',
 'R08',
 'R09',
 'R10',
 'AV01',
 'AV02',
 'AV03',
 'AV04',
 'AV05',
 'AV06',
 'AV07',
 'AV08',
 'AV09',
 'AV10']

In [32]:
columns = [#'index',
 'Date','Ticker','Y',
 'R', 'R01', 'R02', 'R03', 'R04', 'R05', 'R06', 'R07', 'R08', 'R09', 'R10',
 'AV', 'AV01', 'AV02', 'AV03', 'AV04', 'AV05', 'AV06', 'AV07', 'AV08', 'AV09', 'AV10']
dtdf = dtdf[columns]

In [33]:
# Convert Date and Ticker into columns - not necessary if from flatfile
#dtdf.reset_index(inplace=True)
display(dtdf)

,Date,Ticker,Y,R,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,AV,AV01,AV02,AV03,AV04,AV05,AV06,AV07,AV08,AV09,AV10
120,2000-01-13,AAPL,1,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,-0.12,-0.03,-0.07,-0.03,"365,218,945.34","349,298,845.85","331,659,328.97","336,301,291.86","342,881,442.56","360,689,811.63","349,572,083.91","338,073,988.62","338,730,434.62","344,591,098.97","363,007,272.51"
124,2000-01-14,AAPL,1,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,-0.12,-0.03,-0.07,"362,641,843.67","365,218,945.34","349,298,845.85","331,659,328.97","336,301,291.86","342,881,442.56","360,689,811.63","349,572,083.91","338,073,988.62","338,730,434.62","344,591,098.97"
128,2000-01-18,AAPL,1,0.04,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,-0.12,-0.03,"361,926,200.44","362,641,843.67","365,218,945.34","349,298,845.85","331,659,328.97","336,301,291.86","342,881,442.56","360,689,811.63","349,572,083.91","338,073,988.62","338,730,434.62"
132,2000-01-19,AAPL,1,0.09,0.04,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,-0.12,"376,383,516.50","361,926,200.44","362,641,843.67","365,218,945.34","349,298,845.85","331,659,328.97","336,301,291.86","342,881,442.56","360,689,811.63","349,572,083.91","338,073,988.62"
136,2000-01-20,AAPL,1,0.11,0.09,0.04,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,"447,471,279.14","376,383,516.50","361,926,200.44","362,641,843.67","365,218,945.34","349,298,845.85","331,659,328.97","336,301,291.86","342,881,442.56","360,689,811.63","349,572,083.91"
140,2000-01-21,AAPL,0,0.11,0.11,0.09,0.04,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,"456,549,821.86","447,471,279.14","376,383,516.50","361,926,200.44","362,641,843.67","365,218,945.34","349,298,845.85","331,659,328.97","336,301,291.86","342,881,442.56","360,689,811.63"
144,2000-01-24,AAPL,1,0.03,0.11,0.11,0.09,0.04,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,"465,858,524.74","456,549,821.86","447,471,279.14","376,383,516.50","361,926,200.44","362,641,843.67","365,218,945.34","349,298,845.85","331,659,328.97","336,301,291.86","342,881,442.56"
148,2000-01-25,AAPL,1,0.13,0.03,0.11,0.11,0.09,0.04,0.02,-0.00,-0.08,-0.06,-0.05,"481,616,413.84","465,858,524.74","456,549,821.86","447,471,279.14","376,383,516.50","361,926,200.44","362,641,843.67","365,218,945.34","349,298,845.85","331,659,328.97","336,301,291.86"
152,2000-01-26,AAPL,1,0.12,0.13,0.03,0.11,0.11,0.09,0.04,0.02,-0.00,-0.08,-0.06,"488,127,199.61","481,616,413.84","465,858,524.74","456,549,821.86","447,471,279.14","376,383,516.50","361,926,200.44","362,641,843.67","365,218,945.34","349,298,845.85","331,659,328.97"
156,2000-01-27,AAPL,1,0.09,0.12,0.13,0.03,0.11,0.11,0.09,0.04,0.02,-0.00,-0.08,"491,663,343.83","488,127,199.61","481,616,413.84","465,858,524.74","456,549,821.86","447,471,279.14","376,383,516.50","361,926,200.44","362,641,843.67","365,218,945.34","349,298,845.85"


In [34]:
# Find the intersection of minimum and maximum of dates across tickers
# then use this dataframe for all observations

mindate = dtdf.groupby('Ticker')['Date'].min()
maxdate = dtdf.groupby('Ticker')['Date'].max()

begdate = max(mindate)
enddate = min(maxdate)

print begdate, enddate

2004-09-17 2017-06-02


In [35]:
df1 = dtdf.where((dtdf['Date'] >= begdate) & (dtdf['Date'] <= enddate))\
      .dropna(axis=0, how='all')
display(df1)

,Date,Ticker,Y,R,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,AV,AV01,AV02,AV03,AV04,AV05,AV06,AV07,AV08,AV09,AV10
4836,2004-09-17,AAPL,1.00,0.21,0.15,0.14,0.15,0.15,0.18,0.15,0.15,0.18,0.18,0.14,"219,559,758.92","214,147,068.13","208,767,832.07","210,204,598.47","213,725,751.70","213,791,432.11","209,843,079.33","204,623,590.50","203,476,436.62","201,875,425.28","205,360,390.52"
4841,2004-09-20,AAPL,1.00,0.22,0.21,0.15,0.14,0.15,0.15,0.18,0.15,0.15,0.18,0.18,"219,142,114.70","219,559,758.92","214,147,068.13","208,767,832.07","210,204,598.47","213,725,751.70","213,791,432.11","209,843,079.33","204,623,590.50","203,476,436.62","201,875,425.28"
4846,2004-09-21,AAPL,1.00,0.22,0.22,0.21,0.15,0.14,0.15,0.15,0.18,0.15,0.15,0.18,"224,611,086.39","219,142,114.70","219,559,758.92","214,147,068.13","208,767,832.07","210,204,598.47","213,725,751.70","213,791,432.11","209,843,079.33","204,623,590.50","203,476,436.62"
4851,2004-09-22,AAPL,1.00,0.16,0.22,0.22,0.21,0.15,0.14,0.15,0.15,0.18,0.15,0.15,"226,930,891.07","224,611,086.39","219,142,114.70","219,559,758.92","214,147,068.13","208,767,832.07","210,204,598.47","213,725,751.70","213,791,432.11","209,843,079.33","204,623,590.50"
4856,2004-09-23,AAPL,1.00,0.13,0.16,0.22,0.22,0.21,0.15,0.14,0.15,0.15,0.18,0.15,"225,399,146.59","226,930,891.07","224,611,086.39","219,142,114.70","219,559,758.92","214,147,068.13","208,767,832.07","210,204,598.47","213,725,751.70","213,791,432.11","209,843,079.33"
4861,2004-09-24,AAPL,1.00,0.08,0.13,0.16,0.22,0.22,0.21,0.15,0.14,0.15,0.15,0.18,"209,793,465.19","225,399,146.59","226,930,891.07","224,611,086.39","219,142,114.70","219,559,758.92","214,147,068.13","208,767,832.07","210,204,598.47","213,725,751.70","213,791,432.11"
4866,2004-09-27,AAPL,1.00,0.09,0.08,0.13,0.16,0.22,0.22,0.21,0.15,0.14,0.15,0.15,"211,053,920.92","209,793,465.19","225,399,146.59","226,930,891.07","224,611,086.39","219,142,114.70","219,559,758.92","214,147,068.13","208,767,832.07","210,204,598.47","213,725,751.70"
4871,2004-09-28,AAPL,1.00,0.11,0.09,0.08,0.13,0.16,0.22,0.22,0.21,0.15,0.14,0.15,"215,886,136.18","211,053,920.92","209,793,465.19","225,399,146.59","226,930,891.07","224,611,086.39","219,142,114.70","219,559,758.92","214,147,068.13","208,767,832.07","210,204,598.47"
4876,2004-09-29,AAPL,1.00,0.12,0.11,0.09,0.08,0.13,0.16,0.22,0.22,0.21,0.15,0.14,"213,944,096.10","215,886,136.18","211,053,920.92","209,793,465.19","225,399,146.59","226,930,891.07","224,611,086.39","219,142,114.70","219,559,758.92","214,147,068.13","208,767,832.07"
4881,2004-09-30,AAPL,1.00,0.08,0.12,0.11,0.09,0.08,0.13,0.16,0.22,0.22,0.21,0.15,"212,311,312.92","213,944,096.10","215,886,136.18","211,053,920.92","209,793,465.19","225,399,146.59","226,930,891.07","224,611,086.39","219,142,114.70","219,559,758.92","214,147,068.13"


In [36]:
# Convert to "wide format" so that for each day, 
# there are a number of columns,
# and each ticekr's name is atteched to its respective columns
df2 = df1.pivot(index='Date', columns='Ticker')
df2.columns = [' '.join(col).strip() for col in df2.columns.values]
df2.reset_index(inplace=True)

display(df2)

,Date,Y AAPL,Y AMZN,Y GOOGL,Y IBM,Y MSFT,R AAPL,R AMZN,R GOOGL,R IBM,R MSFT,R01 AAPL,R01 AMZN,R01 GOOGL,R01 IBM,R01 MSFT,R02 AAPL,R02 AMZN,R02 GOOGL,R02 IBM,R02 MSFT,R03 AAPL,R03 AMZN,R03 GOOGL,R03 IBM,R03 MSFT,R04 AAPL,R04 AMZN,R04 GOOGL,R04 IBM,R04 MSFT,R05 AAPL,R05 AMZN,R05 GOOGL,R05 IBM,R05 MSFT,R06 AAPL,R06 AMZN,R06 GOOGL,R06 IBM,R06 MSFT,R07 AAPL,R07 AMZN,R07 GOOGL,R07 IBM,R07 MSFT,R08 AAPL,R08 AMZN,R08 GOOGL,R08 IBM,R08 MSFT,R09 AAPL,R09 AMZN,R09 GOOGL,R09 IBM,R09 MSFT,R10 AAPL,R10 AMZN,R10 GOOGL,R10 IBM,R10 MSFT,AV AAPL,AV AMZN,AV GOOGL,AV IBM,AV MSFT,AV01 AAPL,AV01 AMZN,AV01 GOOGL,AV01 IBM,AV01 MSFT,AV02 AAPL,AV02 AMZN,AV02 GOOGL,AV02 IBM,AV02 MSFT,AV03 AAPL,AV03 AMZN,AV03 GOOGL,AV03 IBM,AV03 MSFT,AV04 AAPL,AV04 AMZN,AV04 GOOGL,AV04 IBM,AV04 MSFT,AV05 AAPL,AV05 AMZN,AV05 GOOGL,AV05 IBM,AV05 MSFT,AV06 AAPL,AV06 AMZN,AV06 GOOGL,AV06 IBM,AV06 MSFT,AV07 AAPL,AV07 AMZN,AV07 GOOGL,AV07 IBM,AV07 MSFT,AV08 AAPL,AV08 AMZN,AV08 GOOGL,AV08 IBM,AV08 MSFT,AV09 AAPL,AV09 AMZN,AV09 GOOGL,AV09 IBM,AV09 MSFT,AV10 AAPL,AV10 AMZN,AV10 GOOGL,AV10 IBM,AV10 MSFT
0,2004-09-17,1.00,0.00,1.00,0.00,1.00,0.21,0.11,0.17,0.01,0.01,0.15,0.08,0.07,0.01,-0.01,0.14,0.10,0.06,0.03,0.01,0.15,0.12,0.05,0.03,0.01,0.15,0.11,0.05,0.03,0.01,0.18,0.06,0.08,0.06,0.02,0.15,0.04,0.08,0.03,-0.00,0.15,0.02,0.08,0.01,-0.02,0.18,0.09,0.07,0.02,0.01,0.18,0.09,0.07,0.01,-0.00,0.14,0.10,0.07,-0.01,0.00,"219,559,758.92","295,295,629.57","508,588,884.88","263,741,569.62","873,070,421.48","214,147,068.13","302,882,886.14","3,250,494,371.22","258,640,513.47","854,596,244.17","208,767,832.07","296,993,529.92","3,174,982,857.13","261,047,814.69","876,020,800.35","210,204,598.47","294,829,499.08","3,221,311,366.90","257,153,246.13","879,542,110.67","213,725,751.70","276,458,134.97","3,208,938,125.76","258,064,698.59","877,284,133.29","213,791,432.11","269,322,060.58","3,305,496,494.59","260,827,920.41","872,290,144.11","209,843,079.33","276,929,220.24","3,476,070,785.80","270,901,977.21","868,888,248.06","204,623,590.50","280,958,269.06","3,480,190,122.43","272,912,432.90","865,451,049.19","203,476,436.62","287,074,085.06","3,478,905,651.54","268,223,778.32","870,221,627.50","201,875,425.28","292,031,919.35","3,514,978,060.75","266,772,942.72","876,251,112.86","205,360,390.52","302,333,536.62","3,528,430,379.53","272,257,300.66","903,231,652.31"
1,2004-09-20,1.00,0.00,1.00,1.00,1.00,0.22,0.10,0.10,0.01,0.01,0.21,0.11,0.17,0.01,0.01,0.15,0.08,0.07,0.01,-0.01,0.14,0.10,0.06,0.03,0.01,0.15,0.12,0.05,0.03,0.01,0.15,0.11,0.05,0.03,0.01,0.18,0.06,0.08,0.06,0.02,0.15,0.04,0.08,0.03,-0.00,0.15,0.02,0.08,0.01,-0.02,0.18,0.09,0.07,0.02,0.01,0.18,0.09,0.07,0.01,-0.00,"219,142,114.70","301,833,067.24","478,445,205.49","263,419,386.51","878,373,383.16","219,559,758.92","295,295,629.57","508,588,884.88","263,741,569.62","873,070,421.48","214,147,068.13","302,882,886.14","3,250,494,371.22","258,640,513.47","854,596,244.17","208,767,832.07","296,993,529.92","3,174,982,857.13","261,047,814.69","876,020,800.35","210,204,598.47","294,829,499.08","3,221,311,366.90","257,153,246.13","879,542,110.67","213,725,751.70","276,458,134.97","3,208,938,125.76","258,064,698.59","877,284,133.29","213,791,432.11","269,322,060.58","3,305,496,494.59","260,827,920.41","872,290,144.11","209,843,079.33","276,929,220.24","3,476,070,785.80","270,901,977.21","868,888,248.06","204,623,590.50","280,958,269.06","3,480,190,122.43","272,912,432.90","865,451,049.19","203,476,436.62","287,074,085.06","3,478,905,651.54","268,223,778.32","870,221,627.50","201,875,425.28","292,031,919.35","3,514,978,060.75","266,772,942.72","876,251,112.86"
2,2004-09-21,1.00,0.00,1.00,1.00,1.00,0.22,0.10,0.08,0.01,0.00,0.22,0.10,0.10,0.01,0.01,0.21,0.11,0.17,0.01,0.01,0.15,0.08,0.07,0.01,-0.01,0.14,0.10,0.06,0.03,0.01,0.15,0.12,0.05,0.03,0.01,0.15,0.11,0.05,0.03,0.01,0.18,0.06,0.08,0.06,0.02,0.15,0.04,0.08,0.03,-0.00,0.15,0.02,0.08,0.01,-0.02,0.18,0.09,0.07,0.02,0.01,"224,611,086.39","305,412,423.55","449,781,859.73","262,897,371.99","910,14

In [37]:
# This dataset will form the basis for the machine learning part of the analysis
#df2.to_csv('Predict_01.CSV')